## PROBLEM DESCRIPTION

The data we want to analyze is styles from a group of elements. In this case we had 4 styles so we want at least one machine for every one that it will be able to analyze the data an generate new data.

So the data that we have is a set of vectors that's the output of the app that generate grids. This grids ara a composition of elements so the outuput will be a vector of id. Each grid had 48 elements and each element can have a maximum of 255 elements in it. But is not necessary to had that number, it can be possible to have less. 

<h4 align="center">$[ x_1, x_2, x_3, ..., x_{48} ]$</h4> 
<h4 align="center">$x_1 = [id_1,id_2,_id_3, ...,id_{255}]$</h4>

Each element of the vector can have a different size from the others and the list of integers are the subcategories of style of the object that was placed in the grid. Each number is a different subcategory of style of the object placed previously.

So the first objective is to descompose the input data, a vector which element is a vector of different size, to a list of vectors that will have de same length, 48, and one element in each position. So the main goal is to find the grid wich elements has more IDs so we can set the size of total vectors,$y$, that we need to descompose the data. In case that one element of the vector has lower IDs that the max we gonna put a $0$ as the null value for our data.

<h4 align="center">$input = [[id_1^1,id_2^1,_id_3^1, ...,id_{255}^1], [id_1^2,id_2^2,_id_3^2, ...,id_{255}^2], ..., [id_1^{48},id_2^{48},_id_3^{48}, ...,id_{255}^{48}]]$</h4> 
<h4 align="center">$output = [[id_1^1, id_1^2,..., id_1^{48}],
[id_2^1, id_2^2,..., id_2^{48}],..., [0,0,...,id_y^{48}]$</h4> 

In [1]:
import pandas as pd
import json
import sys
import os
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent =  currentdir + '\RBM'
sys.path.insert(0,parent)
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from timeit import default_timer as timer
import numpy as np
import time
import matplotlib.pyplot as plt
import numexpr  as ne
import profile
import rbm as Rbm
import pandas
from random import randint
import random
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import BernoulliRBM
import Utils

In the file *Utils.py* we have 2 two functions that allow us to transform our input data. 

The first one is *getMaxLengthForElements*. This functions returns a integer that specifies the max length that the vectors must have after analyzing all the input vectors.


The second one is *transformInputVector*. This function transform the input data by transforming each vector to the maximum length and adding $0$ to the vectors who had less length than the maximum.


Applying this 2 functions we can transform our data to be processed and converted to one hot vectors.

In [2]:
vector_data = pd.read_json('./Data.json')
A = vector_data.values
data_modern = []
A = A.ravel()
length_vectors = len(A)
for i in range(A.shape[0]):
    len_v = len(A[i])
    tmp = []
    for j in range(len_v):
        tmp.append(A[i][j])
    tmp = np.asarray(tmp).ravel()
    data_modern.append(tmp)
data_modern = np.asarray(data_modern)
input_data = []
input_data.append(data_modern)
input_data = np.asarray(input_data)
max_elements = Utils.getMaxLengthForElements(input_data[0])
f_data = Utils.transformInputVector(input_data[0], max_elements, length_vectors)

#Multiply data for obtain more range <= 10000.
for i in range(50):
    for k in range((length_vectors*max_elements)):
        f_data.append(f_data[k])
f_data = np.asarray(f_data)
print(f_data.shape)

(6528, 48)


At this poing we have the data formatted ready for the input for the machine. In this case we gonna use the algorithm of the library sklearn, BernoulliRBM. Also we gonna use the implementation of One Hot Vector to transform our data to One Hot Vectors.

http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.BernoulliRBM.html

http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

Now the first step to get the data in the machine is to convert all the data we formatted to one hot vectors. A one hot vector is a vector which is is a group of bits among which the legal combinations of values are only those with a single high (1) bit and all the others low (0).

In [3]:
oneHotEncoder = OneHotEncoder(262, sparse=True).fit(f_data)
oneHotData = oneHotEncoder.transform(f_data)

At this point we have 3 hot vectors which one represents the previous input data. This is a sparse matrix so is fastest to work than a simple vector and it's easy to show the results.
Then we neeed to define the parameters of the machine and fit it with de one hot vectors we had. The parameters are not specified, so we need to try whichc are the best to obtain good results


In [4]:
%%time
RBM_Machine = BernoulliRBM(n_components=262, learning_rate=0.01,batch_size=64, n_iter=20, random_state=0, verbose=True)
RBM_Machine.fit(oneHotData)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -2536.59, time = 11.25s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -2149.61, time = 11.06s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -2060.71, time = 12.07s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -1686.45, time = 11.16s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -1184.61, time = 11.06s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -898.52, time = 11.69s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -731.96, time = 10.62s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -671.00, time = 11.49s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -610.50, time = 10.89s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -545.41, time = 10.80s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -458.54, time = 10.85s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -356.30, time = 11.39s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -255.24, time = 12.47s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -154.78, time = 11.

Finally we got the output. We gonna make as much as the maximum length of samples. Then we gonna format this data and save it to a json in our folder.

In [13]:
final_output = []
for i in range(16):
    x_visible = RBM_Machine.gibbs(oneHotData[random.randint(0,oneHotData.shape[0])])
    x_visible = x_visible.ravel()
    final_vector = []
    for k in range(len(x_visible)):
        if(x_visible[k] == True):
            final_vector.append(1)
        else:
            final_vector.append(0)
    final_Data = []
    for i in range(48):
        tmp = final_vector[(262*i):((i+1)*262)]
        if(1 not in tmp):
            index = 0
        else:
            index = tmp.index(1)
        final_Data.append(index)
    final_output.append(final_Data)
final_output = np.asarray(final_output)
final_data_output = []
pos = 0
for k in range(48):
    t = []
    for i in range(len(final_output)):
        t.append(final_output[i][k])
    final_data_output.append(t)
final_data_output = np.asarray(final_data_output)
output_grid = []
for i in range(48):
    v = final_data_output[i].tolist()
    v_filter = list(filter((0).__ne__,v))
    if(len(v_filter) == 0):
        v_filter.append(0)
    output_grid.append(v_filter)
print(output_grid)

[[218, 205, 229, 252, 14, 220], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [8, 4, 12, 4, 121], [246, 13, 7], [237, 115], [245, 219, 176, 246, 177, 127, 219], [219, 14, 176, 232, 166], [0], [0], [199, 57], [218], [9, 239, 183], [254, 130, 219, 14], [8, 218, 176], [176, 128, 155, 218, 218], [218, 212], [245], [0], [154, 219, 218, 219, 154, 254, 154], [253, 219, 136], [109, 218], [0], [0], [245], [219, 218]]


In [14]:
final_output_grid = Utils.cleaningOutput(output_grid)
print(final_output_grid)

[[14, 205, 218, 220, 229, 252], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [4, 8, 12, 121], [7, 13, 246], [115, 237], [127, 176, 177, 219, 245, 246], [14, 166, 176, 219, 232], [0], [0], [57, 199], [218], [9, 183, 239], [14, 130, 219, 254], [8, 176, 218], [128, 155, 176, 218], [212, 218], [245], [0], [154, 218, 219, 254], [136, 219, 253], [109, 218], [0], [0], [245], [218, 219]]


In [9]:
import codecs, json 
b = final_output_grid
json_file = "./output_grid.json" 
json.dump(b, codecs.open(json_file, 'w', encoding='utf-8'), sort_keys=True, indent=4)

## OTHER IMPLEMENTATION

In [5]:
oneHotEncoder = OneHotEncoder(262, sparse=False).fit(f_data)
oneHotData = oneHotEncoder.transform(f_data)
test_Data_Vector_Aux = np.array(oneHotData, dtype="float64")

In [6]:
visible_dim = oneHotData.shape[1]
hidden_dim = 500
epochs = 100
K = 1
lr = 0.1
batch_size = 10

In [7]:
rbm_ = Rbm.RBM(visible_dim=visible_dim,
               hidden_dim=hidden_dim,
               seed=42,
               mu=0, 
               sigma=0.3,
               monitor_time=True)

In [8]:
%%time
rbm_.fit(test_Data_Vector_Aux, 
         method='vectorized_CDK',
         K=1,
         lr=0.01,
         epochs=20,
         batch_size=64,
         plot_weights=False)

KeyboardInterrupt: 

In [17]:
final_output = []
for i in range(100):
    x_visible, x_h = rbm_.sample_visible_from_visible(oneHotData[random.randint(0,oneHotData.shape[0])])
    x_visible = x_visible.ravel()
    final_vector = []
    for k in range(len(x_visible)):
        if(x_visible[k] == True):
            final_vector.append(1)
        else:
            final_vector.append(0)
    final_Data = []
    for i in range(48):
        tmp = final_vector[(262*i):((i+1)*262)]
        if(1 not in tmp):
            index = 0
        else:
            index = tmp.index(1)
        final_Data.append(index)
    final_output.append(final_Data)
final_output = np.asarray(final_output)
final_data_output = []
for k in range(48):
    t = []
    for i in range(len(final_output)):
        t.append(final_output[i][k])
    final_data_output.append(t)
final_data_output = np.asarray(final_data_output)
output_grid = []
for i in range(48):
    v = final_data_output[i].tolist()
    v_filter = list(filter((0).__ne__,v))
    if(len(v_filter) == 0):
        v_filter.append(0)
    output_grid.append(v_filter)
print(output_grid)

[[245, 218, 218, 247, 121, 205, 205, 4, 220, 121, 245, 248, 4, 247], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [219, 6, 10, 246, 6, 14, 218, 13, 6, 4], [12, 7, 7, 12, 12, 218, 7, 218, 4], [218, 218, 14, 218, 245, 14, 12, 245, 14, 4], [218, 183, 218, 183, 177, 176, 218, 183, 219, 154], [177, 6, 14, 245, 213, 176, 14, 239, 6, 237, 245, 234, 8, 218], [46, 87, 10, 13, 46], [218, 176, 138, 219, 218, 237, 40, 246, 31], [246, 218, 218, 237, 10, 12, 12, 237, 10, 4], [218, 6, 218, 4, 219, 14, 219, 6, 198], [9, 9, 218, 183, 14, 14, 246], [219, 8, 219, 237, 8, 10, 218, 8, 7], [177, 177, 9, 177, 218, 177, 177, 176], [219, 219, 40, 14, 14, 218, 40, 155], [197, 218, 197, 218, 14, 237, 197], [13, 218, 13, 219, 219, 245, 13, 246, 198], [219, 9, 206, 9, 218, 207, 246, 9, 246, 135], [245, 245, 219, 202, 202, 154, 176, 240, 202, 8], [13, 13, 13, 202, 210, 13, 136], [219, 14], [136, 9, 218, 248, 14], [219, 248, 219], [12, 176, 2

In [18]:
final_output_grid = Utils.cleaningOutput(output_grid)
print(final_output_grid)

[[4, 121, 205, 218, 220, 245, 247, 248], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [4, 6, 10, 13, 14, 218, 219, 246], [4, 7, 12, 218], [4, 12, 14, 218, 245], [154, 176, 177, 183, 218, 219], [6, 8, 14, 176, 177, 213, 218, 234, 237, 239, 245], [10, 13, 46, 87], [31, 40, 138, 176, 218, 219, 237, 246], [4, 10, 12, 218, 237, 246], [4, 6, 14, 198, 218, 219], [9, 14, 183, 218, 246], [7, 8, 10, 218, 219, 237], [9, 176, 177, 218], [14, 40, 155, 218, 219], [14, 197, 218, 237], [13, 198, 218, 219, 245, 246], [9, 135, 206, 207, 218, 219, 246], [8, 154, 176, 202, 219, 240, 245], [13, 136, 202, 210], [14, 219], [9, 14, 136, 218, 248], [219, 248], [4, 12, 14, 176, 240, 246], [4, 7, 120, 218, 245, 253]]


In [19]:
b = output_grid
json_file = "./output_grid_2.json" 
json.dump(b, codecs.open(json_file, 'w', encoding='utf-8'), sort_keys=True, indent=4)

###  Evaluation Quantitativa

Evaluar models generatius és en general molt complicat.

Per a decidir que el nostre sistema ¨va bé¨ (va millor que generar coses aleatoriament). S´ha decidit generar

- 25 dades amb la RBM. 
- 25 dades aleatories.

Donem les graelles al expert i li preguntem quines son bones i quines no. Contem quantes "de les bones" corresponnen amb les generades per la RBM.

#### Evaluacio custom

Podriem mirar de les graelles generades si els objectes son de color similar.

###  Evaluation Qualitativa

Posar en el document les graelles generades per la RBM, alguna graella random i alguna graella original. Explicar quins aspectes son bons i quins dolents de les graelles generades. Per exemple, demanar al expert que, de les graelles RBM generades quines coses 'no li agraden'.

### Posar imatge de la eina

Interficie grafica de la imatge.

- Que ap
orta la eina
- Temps que triguen els humans a generar una graella amb photoshop
- Temps que triguen els humans a generar una graella amb eina
- Temps que triguen els humans a generar una graella amb RBM
    - Podries vendre-ho com una "help tool" que a un disenyador li genera 25 graelles i ell sols ha de triar
      quina li agrada més. I si vol variar alguna cosa de la graella.
    
